In [2]:
import os
import pandas as pd
import cv2
import numpy as np
from random import shuffle


In [3]:
train_dir = 'train_data'
df = pd.read_csv('train_labels.csv')
IMG_SIZE = 50
LR = 1e-3

In [4]:
def label_image(img):
    result = df.loc[df['id']==int(img)]['label']
    if result.tolist()[0] == 'cat':
        return 0
    else:
        return 1

In [5]:
def preprocess_training_data():
    training_data = []
    for img_path in os.listdir(train_dir):
        label = label_image(img_path.split('.')[0])
        image = cv2.resize(cv2.imread(os.path.join(train_dir,img_path),cv2.IMREAD_UNCHANGED),(IMG_SIZE,IMG_SIZE))
        training_data.append([image,label])
    shuffle(training_data)
    return training_data

In [6]:
training_data = preprocess_training_data()

In [8]:
X = []
y = []
for features,label in training_data:
    X.append(features)
    y.append(label)

X = np.array(X)/255
y = np.array(y)


In [9]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

In [16]:
model = Sequential()

model.add(Conv2D(64,(3,3),activation='relu'))
model.add(MaxPooling2D(2,2))

model.add(Conv2D(64,(3,3),activation='relu'))
model.add(MaxPooling2D(2,2))

model.add(Conv2D(64,(3,3),activation='relu'))
model.add(MaxPooling2D(2,2))

model.add(Flatten())
model.add(Dense(128,input_shape=X.shape[1:],activation='relu'))
model.add(Dense(128,activation='relu'))

model.add(Dense(2, activation='softmax'))

In [18]:
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [19]:
model.fit(X,y,epochs=5,validation_split=0.1,batch_size=32)

Epoch 1/5
225/225 [==============================] - 25s 109ms/step - loss: 0.6784 - accuracy: 0.5742 - val_loss: 0.6458 - val_accuracy: 0.6062
Epoch 2/5
225/225 [==============================] - 24s 109ms/step - loss: 0.6443 - accuracy: 0.6235 - val_loss: 0.6179 - val_accuracy: 0.6575
Epoch 3/5
225/225 [==============================] - 25s 110ms/step - loss: 0.6061 - accuracy: 0.6683 - val_loss: 0.5925 - val_accuracy: 0.6787
Epoch 4/5
225/225 [==============================] - 24s 107ms/step - loss: 0.5711 - accuracy: 0.7001 - val_loss: 0.5759 - val_accuracy: 0.6988
Epoch 5/5
225/225 [==============================] - 25s 109ms/step - loss: 0.5311 - accuracy: 0.7308 - val_loss: 0.5226 - val_accuracy: 0.7325


In [21]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (32, 48, 48, 64)          1792      
                                                                 
 max_pooling2d_2 (MaxPooling  (32, 24, 24, 64)         0         
 2D)                                                             
                                                                 
 conv2d_3 (Conv2D)           (32, 22, 22, 64)          36928     
                                                                 
 max_pooling2d_3 (MaxPooling  (32, 11, 11, 64)         0         
 2D)                                                             
                                                                 
 conv2d_4 (Conv2D)           (32, 9, 9, 64)            36928     
                                                                 
 max_pooling2d_4 (MaxPooling  (32, 4, 4, 64)          